# LOCALIZATION SYSTEMS – Positioning with Inertial Measurement Units (Practice 3)

To run this practice using Google's collab, follow this link [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/mgfernan/gsl_uab/blob/main/p3_GNSS_IMU.ipynb)

In this practice, we will focus on the fundamentals of positioning based on Inertial Measurement Units (IMU). We will compute the position by integrating the acceleration given by the IMU sensor.


In [1]:
# Add required libraries
import pandas as pd
import matplotlib.pyplot as plt

Download the necessary files for this practice

In [ ]:
# Download necessary files
import requests

for file_name in ['data.csv', 'trajectory.csv']:
  url = f"https://github.com/mgfernan/gsl_uab/raw/main/data/{file_name}"

  response = requests.get(url)
  if response.status_code == 200:
      with open(file_name, 'wb') as file:
          file.write(response.content)
      print(f"File '{file_name}' downloaded successfully.")
  else:
      print(f"Failed to download the file. Status code: {response.status_code}")

## Short tutorial on `pandas`

Pandas is a powerful Python library for data manipulation and analysis. It provides data structures and functions needed to work with structured data seamlessly. In this tutorial, we'll cover some fundamental aspects of Pandas.

### Creating a DataFrame

A DataFrame is a two-dimensional labeled data structure with columns that can be of different types. You can create a DataFrame using various methods, such as from dictionaries, lists, or reading data from files.

In [ ]:
# Create a DataFrame from a dictionary
data = {'Name': ['Alice', 'Bob', 'Charlie'],
        'Age': [25, 30, 22]}
df = pd.DataFrame(data)

# Display the DataFrame
print(df)


### Reading Data from a CSV File

Pandas makes it easy to read data from various file formats, such as CSV, Excel, SQL databases, etc. Here's how you can read data from a CSV file.

**Note**: You will have to upload the [`data.csv` file](https://raw.githubusercontent.com/mgfernan/gsl_uab/main/data/data.csv) from the Github repository to the Colab workspace  

In [ ]:
# Read data from a CSV file
file_path = 'data.csv'
df = pd.read_csv(file_path)

# Display the first few rows of the DataFrame
print(df.head())


### Basic DataFrame Operations

Pandas offers numerous functions to manipulate and analyze data:

In [ ]:
# Selecting a column
ages = df['Age']
print(ages)

In [ ]:
# Filtering rows
young_people = df[df['Age'] < 30]
print(young_people)

In [ ]:
# Adding a new column
df['Gender'] = ['F', 'M', 'M', 'M', 'F']
print(df)

In [ ]:
# Grouping and aggregation
grouped = df.groupby('Gender').mean()
print(grouped)

In [ ]:
# Sorting
sorted_df = df.sort_values(by='Age', ascending=False)
print(sorted_df)

### Data Visualization

Albeit Pandas also provides basic plotting capabilities using the `plot()` function, you can also use `matplotlib` for this
purpose, which will give you more flexibility

In [ ]:
# Plotting a bar chart
plt.bar(df['Name'], df['Age'])

## Practice exercises

**Exercise 1: data load**

**Using pandas, read the file `trajectory.csv` that contains the position and acceleration into a `DataFrame` structure**

![Trajectory](./assets/trajectory.png)

**Tip**: Use the Pandas `read_csv` method

Answer the following questions:

1. What are the magnitudes (columns) contained in the `DataFrame`? Describe them briefly in one short sentence.
1. How many epochs does the file contain?
1. What is the sampling rate of the data file?

**Exercise 2: plotting**

**Use matplotlib to plot the horizontal (i.e. North vs East) trajectory**

**Tip:** Use the `plt.plot` method

Answer the following questions:

1. What do you see in the plot?
1. How long is the trajectory in the East component, approximately? And in the North component?



**Exercise 3: Availability and Continuity**

Answer the following questions:

1. Is the trajectory **available** all the time? If not, what are the typical causes for lack of availability in a positioning system based on GNSS? How can it be solved?
1. Compute the **availability** (in percentage) of the position? Tip: you will need to check the number of positions (either East or North) that are valid (i.e. not NaN) using the pandas method `isna()`.
1. Plot the time difference for the valid points. Can this be related to the trajectory plot? How many data gaps do you have in the trajectory? which sizes approximately? Tip: compute the epoch difference for the valid points

**Exercise 4: Trajectory reconstruction**

In this exercise we will use the acceleration to reconstruct the position at the missing parts, just in the same way as the IMU help GNSS in parts of the route where GNSS is not available.

For this exercise you will need the following method

In [ ]:
import math

def integrate_trajectory(epochs, positions, accelerations):

    dts = epochs.diff()

    out = []
    for dt, pos, accel in zip(dts, positions, accelerations):

        if math.isnan(pos):
            prev_velocity = (out[-1] - out[-2]) / dt
            velocity = prev_velocity + accel * dt
            position = position + velocity * dt

        else:
            position = pos

        out.append(position)

    return out


Answer the following questions:

1. Have a look at the method `integrate_trajectory` and briefly describe what it does
1. Plot the acceleration in one of the components. Do we have gaps in this magnitude? Which is the **availability** of the acceleration?
1. Compute the velocity at each point and plot it.
1. Reconstruct (integrate) the trajectory with gaps to achieve 100% availability. Tip: use the `integrate_trajectory` method

**Exercise 4: Trajectory reconstruction with biased accelerations**

One of the main problems of IMU are the biases of the sensors, that need to be corrected during integration, otherwise
the trajectory will be wrong as the errors accumulate in time.

To check this effect, create two new acceleration time series with a constant bias of $0.1 m/s^2$ for the East component and $0.2 m/s^2$ for the North component. Repeat the steps 2 and 4 of Exercise 3 and describe what do you see in the resulting plot (and the reason why)